<a href="https://colab.research.google.com/github/prachirathod99/Text_clustering_algorithms/blob/main/Text_clustering_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Word2vec

In [7]:
#importing dependent libraries
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
assert all([nltk.download('stopwords'), nltk.download('punkt')])==True
from sklearn import cluster
from sklearn import metrics
import numpy as np
import io


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#loading the file from computer for google colab
from google.colab import files
uploaded = files.upload()

Saving combined_data_webex_teams_zoom.csv to combined_data_webex_teams_zoom.csv


In [6]:
uploaded

{'combined_data_webex_teams_zoom.csv': b'key,value,TF-IDF,key_plus_value,value_plus_key\r\nfile,be unpacked,0.182538,file be unpacked,be unpacked file\r\naudio boards,"14.4-kbit/s version, 28.8-kbit/s version",0.145608,"audio boards 14.4-kbit/s version, 28.8-kbit/s version","14.4-kbit/s version, 28.8-kbit/s version audio boards"\r\npasswords,"the end user, the intended purpose",0.139616,"passwords the end user, the intended purpose","the end user, the intended purpose passwords"\r\nwhiteboarding,"desktop sharing, file transfers",0.138964,"whiteboarding desktop sharing, file transfers","desktop sharing, file transfers whiteboarding"\r\nconferences,up to 15 video participants,0.131616,conferences up to 15 video participants,up to 15 video participants conferences\r\nexpiration,of ten covered by password,0.128222,expiration of ten covered by password,of ten covered by password expiration\r\napplication sharing,"shared whiteboard markup, file transfer",0.124034,"application sharing shared 

In [9]:
#creating the dataframe from the uploaded google colab
df = pd.read_csv(io.BytesIO(uploaded['combined_data_webex_teams_zoom.csv']))
df

,key,value,TF-IDF,key_plus_value,value_plus_key
0,file,be unpacked,0.182538,file be unpacked,be unpacked file
1,audio boards,"14.4-kbit/s version, 28.8-kbit/s version",0.145608,"audio boards 14.4-kbit/s version, 28.8-kbit/s ...","14.4-kbit/s version, 28.8-kbit/s version audio..."
2,passwords,"the end user, the intended purpose",0.139616,"passwords the end user, the intended purpose","the end user, the intended purpose passwords"
3,whiteboarding,"desktop sharing, file transfers",0.138964,"whiteboarding desktop sharing, file transfers","desktop sharing, file transfers whiteboarding"
4,conferences,up to 15 video participants,0.131616,conferences up to 15 video participants,up to 15 video participants conferences
...,...,...,...,...,...
295,The record diameters,"nominally 300 mm, 250 mm, 175 mm",0.014835,"The record diameters nominally 300 mm, 250 mm,...","nominally 300 mm, 250 mm, 175 mm The record di..."
296,the transmission mode,300-millisecond leader tone,0.013689,the transmission mode 300-millisecond leader tone,300-millisecond leader tone the transmission mode
297,cloud seeding,"23 users,large-scale recovery",0.009075,"cloud seeding 23 users,large-scale recovery","23 users,large-scale recovery cloud seeding"
298,headphone volumes,their normal loudness,0.009075,headphone volumes their normal loudness,their normal loudness headphone volumes


In [24]:
df.key_plus_value

0                                       file be unpacked
1      audio boards 14.4-kbit/s version, 28.8-kbit/s ...
2           passwords the end user, the intended purpose
3          whiteboarding desktop sharing, file transfers
4                conferences up to 15 video participants
                             ...                        
295    The record diameters nominally 300 mm, 250 mm,...
296    the transmission mode 300-millisecond leader tone
297          cloud seeding 23 users,large-scale recovery
298              headphone volumes their normal loudness
299    audio boards 14.4-kbit/s version, 28.8-kbit/s ...
Name: key_plus_value, Length: 300, dtype: object

In [25]:
split_words = df.key_plus_value.str.split(' ')

In [26]:
split_words

0                                   [file, be, unpacked]
1      [audio, boards, 14.4-kbit/s, version,, 28.8-kb...
2      [passwords, the, end, user,, the, intended, pu...
3      [whiteboarding, desktop, sharing,, file, trans...
4         [conferences, up, to, 15, video, participants]
                             ...                        
295    [The, record, diameters, nominally, 300, mm,, ...
296    [the, transmission, mode, 300-millisecond, lea...
297    [cloud, seeding, 23, users,large-scale, recovery]
298        [headphone, volumes, their, normal, loudness]
299    [audio, boards, 14.4-kbit/s, version,, 28.8-kb...
Name: key_plus_value, Length: 300, dtype: object

In [29]:
for i in split_words:
  print(i)

['file', 'be', 'unpacked']
['audio', 'boards', '14.4-kbit/s', 'version,', '28.8-kbit/s', 'version']
['passwords', 'the', 'end', 'user,', 'the', 'intended', 'purpose']
['whiteboarding', 'desktop', 'sharing,', 'file', 'transfers']
['conferences', 'up', 'to', '15', 'video', 'participants']
['expiration', 'of', 'ten', 'covered', 'by', 'password']
['application', 'sharing', 'shared', 'whiteboard', 'markup,', 'file', 'transfer']
['scores', 'players', 'ten', 'displayed', 'onscreen']
['special', 'gateways', 'VoIP', 'and', 'video', 'conference', 'clients']
['The', 'maximum', 'imprisonment', 'violations,', 'the', 'Computer', 'Fraud', 'and', 'Abuse', 'Act']
['video', 'A', 'broadband', 'speed,', '2', 'Mbit/s']
['password', 'login', 'time', 'zone', 'or', 'IP', 'address', 'verification']
['password', 'of', 'ten', 'via', 'privilege']
['The', 'web', 'based', 'platform', 'hosts', 'up', 'to', '5,000', 'participants']
['meetings', 'up', 'to', '100', 'devices,', 'a', '40-minute', 'time', 'restriction']
['

In [28]:
sum([len(i) for i in split_words])

2066

In [41]:
#removing stopwords and numbers from the sentences
english_stop_words = stopwords.words('english')
def preprocess_data(__texts__):
  __texts__ = __texts__.lower()
  __texts__ = re.sub('[^a-zA-Z]', ' ', __texts__ )
  __texts__ = re.sub(r'\s+', ' ', __texts__)

  # Preparing the dataset
  #all_sentences = nltk.sent_tokenize(__texts__)
  all_words = [nltk.word_tokenize(word) for word in __texts__.split(' ')]
  # Removing Stop Words
  for i in range(len(all_words)):
      all_words[i] = [w for w in all_words[i] if w not in english_stop_words]
  return all_words

In [ ]:
#converting column in a df to string
#dataframe['column_name']=dataframe['column_name'].apply(str)